## Goal
As a first pass at developing a machine learning model to predict California wildfires I will evaluate and tune several different gradient boosting algorithms. The procedure will be as follows:

1. **Classifier selection:** Find the best classification algorithm for the data based on F1 score compute time and memory requirements
2. **Metaparameter optimization:** Optimize data metaparameters such as rolling window width
3. **Class imbalance:** Generate a strategy to deal with the unbalanced nature of the classes
4. **Hyperparameter optimization:** Tune model hyperparameters
5. **Feature importance:** Investigate feature importance and possibly trim/apply dimensionality reduction techniques to the data
6. **Model robustness:** How does the model perform after repeated sampling? Are scores stable?

There are two anticipated issues which will need to be dealt with first:

1. Large dataset size - current working dataset has 7.3 million observations of 25 variables and this is likely to grow as the project progresses
2. Highly imbalanced data (~20 times more observations without fire than with)

Future goals are to add several more factors from various data sources including: elevation, population density, time since last fire and total fires.

In [1]:
# Imports and notebook setup

import random

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from time import time
from statistics import mean
from memory_profiler import memory_usage
from scipy.interpolate import griddata

from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV

from sklearn.metrics import make_scorer
from sklearn.metrics import f1_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix

from sklearn.utils.fixes import loguniform

%matplotlib inline
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})

In [2]:
# Variables

raw_data_file = '../data/stratified_training_data/1992-2015_training_data_raw_n100000_ks_pval0.3.1.csv'

rand_seed = 42
classifier_jobs = 4
optimization_jobs = 2
max_jobs = classifier_jobs * optimization_jobs

sample_size = 10000
repetitions = 5
train_test_split_ratio = 0.3
cv_folds = 5
n_doublings = 8
scoring_func = make_scorer(average_precision_score)
scoring_func_name = "Average precision"
grid_search_iterations = 500

plot_grid_resolution = 500
contourf_levels = 500

weather_variables = [
    'weather_bin_month',
    'weather_bin_year',
    'air.sfc',
    'rhum.2m',
    'dpt.2m',
    'pres.sfc',
    'uwnd.10m', 
    'vwnd.10m',
    'veg',
    'lat',
    'lon',
    'ignition'
]

In [3]:
# Functions

def k_random_sample(data, k):
    # Takes a data frame and an number of observations
    # returns dataframe containing k from n pseudorandom
    # observations with out replacement
    
    n = len(data)
    indices = random.sample(range(0, n), k)
    return data.iloc[indices]

def stratified_sample(data, n):
    # takes a datafram and a sample size n, returns
    # n observations with positive and negative class
    # frequency matched to orignal data
    
    # split positive and negative datsets up
    ignitions = data[data['ignition'] == 1]
    no_ignitions = data[data['ignition'] == 0]
    
    # Calculate ignition & no ignition sample sizes
    ignition_fraction = len(ignitions) / len(data)
    ignition_sample_size = int((n * ignition_fraction))
    no_ignition_sample_size = int((n * (1 - ignition_fraction)))
    
    # sample data
    no_ignitions_sample = k_random_sample(no_ignitions, no_ignition_sample_size)
    ignitions_sample = k_random_sample(ignitions, ignition_sample_size)

    # combine
    sampled_data = no_ignitions_sample.append(ignitions_sample)
    
    return sampled_data

def cross_validate_classifier(classifier, X_train, y_train, folds, scoring_func):
    # Takes a classifier, x and y training data, a number of folds for
    # cross validation and a scoring function. Runs cross validation and returns
    # array of scores from each fold
    
    cv = StratifiedKFold(n_splits=folds)
    cross_val_scores = cross_val_score(classifier, X_train, y_train, scoring=scoring_func, cv=cv)
    
    return cross_val_scores

def fit_model(classifier, X, y):
    # Takes classifier, data and labels, returns
    # fit model
    classifier.fit(X, y)
    return classifier

def display_confusion_matrix(classifier, class_names, x_test, y_test):
    # Takes a fit classifier, the class names as a list and
    # test data. Prints raw confusion matrix and plots
    # normalized confusion matrix

    raw_cm = confusion_matrix(y_test, classifier.predict(x_test))
    print("Raw count confusion matrix")
    print(raw_cm)
    
    normalized_cm = plot_confusion_matrix(
        classifier,
        x_test,
        y_test,
        display_labels=class_names,
        cmap=plt.cm.Blues,
        normalize='true'
    )

    normalized_cm.ax_.set_title("Normalized confusion matrix")

    plt.show()
    
def calc_false_neg_pos_rate(model, x_test, y_test):
    # Takes a fit model and test data, returns 
    # false positive and negative rates
    
    cm = confusion_matrix(y_test, model.predict(x_test))

    TN = cm[0][0]
    FN = cm[1][0]
    FP = cm[0][1]

    false_neg_rate = FN / (FN + TN)
    false_pos_rate = FP / (FP + TN)
    
    return false_neg_rate, false_pos_rate

def scale_weather_variables(weather_variables, X_train, X_test):
    # Uses StandardScaler to convert data to Z-score. Takes list of
    # weather variable names to scale and train/test data. Calculates
    # transformation from test data, applies same transform to
    # train and test data, returns scaled data.

    scaler = StandardScaler()
    scaler.fit(X_train[weather_variables])

    pd.options.mode.chained_assignment = None  # default='warn'
    
    X_train[weather_variables] = scaler.transform(X_train[weather_variables])
    X_test[weather_variables] = scaler.transform(X_test[weather_variables])
    
    pd.options.mode.chained_assignment = 'warn'  # default='warn'
    
    return X_train, X_test

def make_train_test_sample(dataset, sample_size, train_test_split_ratio, rand_seed):
    # Takes data in dataframe, sample size, train test split
    # ratio and random seed. Samples n datapoints from dataset and then
    # runs stratified train test split on sample.
    # return stratified train test split data
    
    column_names = dataset.columns
    
    sampled_data = stratified_sample(dataset, sample_size)

    y = sampled_data['ignition']
    X = sampled_data.drop('ignition', axis=1)

    X_train, X_test, y_train, y_test = train_test_split(
        X,
        y, 
        test_size=train_test_split_ratio, 
        random_state=rand_seed, 
        stratify=y
    )

    return X_train, X_test, y_train, y_test

def FP_rate_scorer(y_test, y_pred):
    # Calulates false positive rate from test data and
    # predictions. For use with make_scorer
    cm = confusion_matrix(y_test, y_pred)
    TN = cm[0][0]
    FP = cm[0][1]
    false_pos_rate = FP / (FP + TN)
    
    return false_pos_rate

def FN_rate_scorer(y_test, y_pred):
    # Calulates false positive rate from test data and
    # predictions. For use with make_scorer
    cm = confusion_matrix(y_test, y_pred)
    TN = cm[0][0]
    FN = cm[1][0]
    false_neg_rate = FN / (FN + TN)
    
    return false_neg_rate

def tune_class_weight(
    class_weights,
    max_jobs,
    rand_seed,
    data,
    repitions
):
    # Takes class weight list, number of parallel jobs
    # A random seed and training data. Itterates over
    # class weights. Stores and returns F1 score,
    # false positive and false negative rates for each
    # class weight in a data frame.
    
    model_scores_columns = [
        'Class weight',
        'CV {} score'.format(scoring_func_name),
        'CV False positive rate',
        'CV False negative rate'
    ]
    
    model_scores = pd.DataFrame(columns=model_scores_columns)

    for class_weight in class_weights:
        
        for i in range(repitions):
            
            # draw new stratifited sample and run train test split
            X_train, X_test, y_train, y_test = make_train_test_sample(
                data_daily_mean, 
                sample_size, 
                train_test_split_ratio, 
                rand_seed
            )

            # Standardize weather variables
            X_train, X_test = scale_weather_variables(weather_variables, X_train, X_test)

            classifier = XGBClassifier(
                n_jobs = max_jobs,
                scale_pos_weight = class_weight,
                random_state = rand_seed
            )

            classifier.fit(X_train, y_train)

            cross_val_scores = cross_validate_classifier(
                classifier, 
                X_train, 
                y_train, 
                cv_folds, 
                scoring_func
            )

            score = mean(cross_val_scores)

            cross_val_FP_rates = cross_validate_classifier(
                classifier, 
                X_train, 
                y_train, 
                cv_folds, 
                make_scorer(FP_rate_scorer)
            ) 

            cross_val_FP_rate = mean(cross_val_FP_rates)

            cross_val_FN_rates = cross_validate_classifier(
                classifier, 
                X_train, 
                y_train, 
                cv_folds, 
                make_scorer(FN_rate_scorer)
            ) 

            cross_val_FN_rate = mean(cross_val_FN_rates)

            model_scores = model_scores.append(pd.Series([
                class_weight,
                np.round(score,3), 
                np.round(cross_val_FP_rate,3), 
                np.round(cross_val_FN_rate,3)
            ], index=model_scores.columns), ignore_index=True)

    return model_scores

def tune_hyperparameters(
    classifier,
    param_dist, 
    X_train, 
    y_train, 
    num_jobs, 
    search_iterations, 
    scoring_func
):
    # Tunes arbitraty hyperparamter(s) takes a classifier
    # a parameter distribution as a dictionary, training data
    # a number of parallel jobs, a number of search iterations
    # and a scoring function to use for cross validation
    # returns winning model and cross validation results as
    # data frame

    random_search = RandomizedSearchCV(
        classifier, 
        param_distributions=param_dist,
        scoring=scoring_func,
        n_iter=search_iterations,
        n_jobs=num_jobs
    )

    best_model = random_search.fit(X_train, y_train)
    
    return best_model, random_search

def regularize_grid(x, y, z, resolution):
    # Takes three coordinate grids and a resolution
    # interpolates and resamples so frequencys match
    # returns regularized data. For use in
    # constructing heatmaps

    # target grid to interpolate to
    xi = np.arange(min(x), max(x), ((max(x) - min(x)) / resolution))
    yi = np.arange(min(y), max(y), ((max(y) - min(y)) / resolution))
    xi, yi = np.meshgrid(xi, yi)

    # interpolate
    zi = griddata((x, y), z, (xi, yi), method='linear')
    
    return xi, yi, zi
    
def plot_relative_feature_importance(model, data, x_test, x_tick_size):
    # Takes a fit model, orignal data and test data, plots relative feature
    # importance
    
    column_names = x_test.columns
    
    if 'weather_bin_time' in column_names:
        x_test = x_test.drop('weather_bin_time', axis=1)
    
    importances = model.feature_importances_
    indices = np.argsort(importances)[::-1]
    feature_names = np.array(list(x_test))

    plt.figure(figsize=(20,10))
    plt.rc('axes', titlesize=30)     # fontsize of the axes title
    plt.rc('axes', labelsize=30)    # fontsize of the x and y labels
    plt.rc('xtick', labelsize=x_tick_size)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=25)    # fontsize of the tick labels
    plt.title("Feature importance")
    plt.bar(range(x_test.shape[1]), importances[indices],
           color="darkblue", align="center")
    plt.xticks(np.arange(len(indices)), feature_names[indices], rotation='vertical')
    plt.xlim([-1, x_test.shape[1]])
    plt.xlabel("Feature")
    plt.ylabel("Relative importance")
    
    plt.show()
    
    sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})

## Data preparation

In [4]:
# Load data
raw_data = pd.read_csv(raw_data_file, low_memory=False)
raw_data = raw_data[weather_variables]

# Shuffel row order
raw_data = raw_data.sample(frac=1).reset_index(drop=True)

raw_data.describe()

,weather_bin_month,weather_bin_year,air.sfc,rhum.2m,dpt.2m,pres.sfc,uwnd.10m,vwnd.10m,veg,lat,lon,ignition
count,56718.000000,56718.000000,56718.000000,56718.000000,56718.000000,56718.000000,56718.000000,56718.000000,56718.000000,56718.000000,56718.000000,56718.000000
mean,6.522004,2003.488857,289.088516,49.092721,274.747569,91521.722452,1.056332,0.149949,30.654752,37.131682,-119.522121,0.045964
std,3.429307,6.931885,13.754099,26.129440,7.382174,6937.239898,2.699176,3.278275,22.749310,2.564622,2.503922,0.209409
min,1.000000,1992.000000,245.539960,1.000000,198.476650,67680.950000,-12.885849,-15.892535,0.000000,32.683890,-124.340800,0.000000
25%,4.000000,1997.000000,279.103145,26.682617,269.964270,87154.098750,-0.729976,-2.051666,8.200000,34.990290,-121.470900,0.000000
50%,7.000000,2003.000000,286.778900,45.410180,275.309430,92312.085000,1.047374,0.112423,30.600000,36.837940,-119.905100,0.000000
75%,9.000000,2010.000000,297.377017,71.062552,280.183288,97031.256250,2.739596,2.255397,43.600000,39.304180,-117.411700,0.000000
max,12.000000,2015.000000,334.044430,100.030075,296.415040,102960.590000,15.625641,22.613800,83.700000,41.971820,-114.364500,1.000000


## Classifier selection: kitchen sink approach

First step is to throw a bunch of different classifiers with default settings at the problem and see how they do. See list below for contenders.

We will start with a small sample of data to quickly get a sense of how well each classifier works. Hopefully we can discard some to make test run times shorter in the next round.

In [5]:
# SKLearn classifiers

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB

# List of classifier descriptions for plotting / dataframes
classifier_descriptions = [
    'Logit. regres.',
    'Decision tree',
    'K neighbors',
    'Rand. forest',
    'QDA',
    'XGBoost',
    'CatBoost',
    'LightGBM',
    'AdaBoost',
    'Linear SVM',
    'RBF SVM',
    'Naive Bayes'
]

# Classifiers with parameters. Set parallism and rand_seed if avalible.
# Also set 'rule of thumb' class weight if avalible for that classifier.

num_negative_observations = len(raw_data[raw_data['ignition'] == 0])
num_positive_observations = len(raw_data[raw_data['ignition'] == 1])
class_ratio = num_negative_observations / num_positive_observations

class_weight_dict = {
    0: (1 / class_ratio),
    1: 1 - (1 / class_ratio)
}

classifiers = (
    
    LogisticRegression(
        n_jobs = max_jobs, 
        random_state = rand_seed,
        class_weight = class_weight_dict
    ),
    
    DecisionTreeClassifier( 
        random_state = rand_seed,
        class_weight = class_weight_dict
    ),
    
    KNeighborsClassifier(
        n_jobs = max_jobs        
    ),
    
    RandomForestClassifier(
        n_jobs = max_jobs,
        class_weight = class_weight_dict,
        random_state = rand_seed
    ),
    
    QuadraticDiscriminantAnalysis(),
    
    XGBClassifier(
        n_jobs = max_jobs,
        scale_pos_weight = class_ratio,
        random_state = rand_seed
    ),
    
    CatBoostClassifier(
        thread_count = max_jobs, 
        silent = True,
        scale_pos_weight = class_ratio,
        random_state = rand_seed
    ),
    
    LGBMClassifier(
        n_jobs = max_jobs,
        scale_pos_weight = class_ratio,
        random_state = rand_seed
    ),
    
    AdaBoostClassifier(random_state = rand_seed),
    
    SVC(kernel = "linear",
        class_weight = 'balanced',
        random_state = rand_seed
    ),
    
    SVC(
        class_weight = 'balanced',
        random_state = rand_seed
    ),
    
    GaussianNB(),
)

In [ ]:
# Loop over classifier list, run k-fold cross validation on stratified 
# training sample for each. Report mean and SD of score, run time and peak memory use

# Start empty dataframe to hold results
classifier_data = pd.DataFrame(columns=[
    'Classifier',
    'Sample size',
    'CV folds',
    'Raw scores',
    'Mean {} score'.format(scoring_func_name),
    '{} score SD'.format(scoring_func_name),
    'Peak memory (GB)',
    'Run time (min.)'
])

# Loop over classifiers
for classifier, description in zip(classifiers, classifier_descriptions):
    
    dTs = []
    max_mems = []
    raw_scores = []
    
    for i in range(repetitions):
        
        # Take stratified sample data of data and do stratified train_test split    
        X_train, X_test, y_train, y_test = make_train_test_sample(
            raw_data, 
            sample_size, 
            train_test_split_ratio, 
            rand_seed
        )
        
        start = time()

        # Cross validate classifier, tracking memory usage
        mem_usage, cross_val_scores = memory_usage((cross_validate_classifier, (
            classifier, 
            X_train, 
            y_train, 
            cv_folds, 
            scoring_func)
        ), retval=True)

        dT = np.round(((time() - start)/60),2)
        dTs.append(dT)
        raw_scores = np.concatenate((raw_scores, cross_val_scores))
        max_mems = np.concatenate((max_mems, mem_usage))

    mean_score = np.round(mean(raw_scores),3)
    sd_score = np.round(np.std(raw_scores),4)
    max_mem = np.round((max(max_mems) / 1000),3)
    
    # Store run results in dataframe
    classifier_data = classifier_data.append(pd.Series([
        description,
        sample_size,
        cv_folds,
        cross_val_scores,
        mean_score,
        sd_score,
        max_mem,
        dT
    ], index=classifier_data.columns), ignore_index=True)
    
#     print("Finished: {}".format(description))
#     print(raw_scores, mean_score)

# Sort results by score
classifier_data = classifier_data.sort_values(['Mean {} score'.format(scoring_func_name)], ascending=[0])

In [ ]:
# Plot comparison of classifiers using mean score
fig = plt.figure(1, figsize=(12, 5))
ax = fig.add_subplot(111)

bp = ax.boxplot(classifier_data['Raw scores'])
ax.set_xticklabels(classifier_data['Classifier'])
plt.xlabel('Classifier')
plt.ylabel('{} score'.format(scoring_func_name))
plt.title('Classifier comparision, 5-fold cross validation')
plt.xticks(rotation=45)
plt.show()

In [ ]:
classifier_data.drop('Raw scores', axis=1)

Not surprisingly, our top three winners are gradient boosting algorithms. All three are within a standard deviation of each other, but CatBoost takes ~15 times longer than the other two. Let's focus on the top three and quickly check how well they will scale as the dataset grows.

## Classifier selection: resource requirements

In [ ]:
# Reduced classifier set

classifier_descriptions = [
    'LightGBM',
    'CatBoost',
    'XGBoost'
]

classifiers = (
    
    LGBMClassifier(
        n_jobs = max_jobs,
        scale_pos_weight = class_ratio,
        random_state = rand_seed
    ),
    
    CatBoostClassifier(
        thread_count = max_jobs, 
        silent = True,
        scale_pos_weight = class_ratio,
        random_state = rand_seed
    ),
    
    XGBClassifier(
        n_jobs = max_jobs,
        scale_pos_weight = class_ratio,
        random_state = rand_seed
    ),
)

In [ ]:
sample_size_data = pd.DataFrame(columns=[
    'Classifier',
    'Sample size',
    'Peak memory (GB)',
    'Run time (min.)'
])

size_of_sample = 5000

# Loop over range n, doubling the size of the dataset sample each time
for i in range(0,(n_doublings+1)):
    
    # Loop over each classifier, store max memory and run time
    for classifier, description in zip(classifiers, classifier_descriptions):
    
        for i in range(repetitions):
            
            X_train, X_test, y_train, y_test = make_train_test_sample(
                raw_data, 
                size_of_sample, 
                train_test_split_ratio, 
                rand_seed
            )
        
            X_train, X_test = scale_weather_variables(weather_variables, X_train, X_test)
            
            start = time()
            mem_usage, model = memory_usage((fit_model, (classifier, X_train, y_train)), retval=True)
            dT = np.round(((time() - start)/60),2)

            max_mem = np.round((max(mem_usage) / 1000),3)

            sample_size_data = sample_size_data.append(pd.Series([
                description,
                size_of_sample,
                max_mem,
                dT
            ], index=sample_size_data.columns), ignore_index=True)
    
    size_of_sample = size_of_sample * 2

In [ ]:
ax = sns.lineplot(x='Sample size', y='Run time (min.)', hue='Classifier', data=sample_size_data)
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.title('Run time vs. data sample size')
plt.show()

In [ ]:
ax = sns.lineplot(x='Sample size', y='Peak memory (GB)', hue='Classifier', data=sample_size_data)
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.title('Peak memory vs. data sample size')
plt.show()

Looks like XGBoost is the holistic winner. XGBoost's cross validation scores are close to those of CatBoost and it seems to be significantly faster. It does appear to use slightly more memory, hopefully that won't be a problem. Let's look at it in a bit more detail:

In [ ]:
# Retrain default model with standard data sample size
model = XGBClassifier(
    n_jobs = max_jobs,
    scale_pos_weight = class_ratio,
    random_state = rand_seed
)

X_train, X_test, y_train, y_test = make_train_test_sample(
    raw_data, 
    sample_size, 
    train_test_split_ratio, 
    rand_seed
)

X_train, X_test = scale_weather_variables(weather_variables, X_train, X_test)

model.fit(X_train, y_train)

# Plot confusion matrix
class_names = np.array(['No ignition', 'Ignition'])
display_confusion_matrix(model, class_names, X_test, y_test)

In [ ]:
# Calculate false positive and negative rates
false_neg_rate, false_pos_rate = calc_false_neg_pos_rate(model, X_test, y_test)
print("False negative rate: {}".format(np.round(false_neg_rate, 2)))
print("False positive rate: {}".format(np.round(false_pos_rate, 2)))

Given the nature of the problem, false negative are more dangerous than false positives. Although our false negative rate is under 10%, even with this small sample, we missed a significant number of fires. Needless to say, there is room for improvement here.

## Metaparameter optimization: to smooth or not to smooth

You may have notices that I loaded three different datasets at the start of this notebook:

1. **raw_data:** No significant manipulations beyond collating the various datasets. Has a time resolution of 3 hr. due to the frequency of the original NOAA weather data set.
2. **data_rolling_window:** Data was reversed with regard to time and then averaged with a 6 hr. rolling window. This has the effect of assigning the window mean to the time bin at the left of the window. This was done because logically we should care more about weather conditions before the fire was discovered than after.
3. **data_daily_mean:** Rolling window data was averaged by day.

**Note:** All averaging includes the boolean ignition value. After averaging is complete any noxel with a non-zero ignition value is reassigned a 1. This tends to make the fires discovery 'spread' to earlier times. Again, this is acceptable and even desirable because logically the fire must have started before it was discovered.

For more details about these three datasets take a look at the [data preprocessing notebook](https://github.com/gperdrizet/wildfire/blob/master/notebooks/preprocess_data.ipynb).

OK, so, let's compare the three sets with cross validation.

In [ ]:
# Roll data with windowed mean
data_rolling_window = raw_data.iloc[::-1]
data_rolling_window = data_rolling_window.groupby(['lat', 'lon']).rolling(48, on="weather_bin_time").mean()
data_rolling_window.reset_index(drop=True, inplace=True)

# Reassign fractional ignition values a '1'
ignitions = data_rolling_window[data_rolling_window['ignition'] > 0].copy()
no_ignitions = data_rolling_window[data_rolling_window['ignition'] == 0].copy()
ignitions['ignition'] = 1

data_rolling_window = ignitions.append(no_ignitions)

In [ ]:
# Average data by day
data_daily_mean = data_rolling_window.groupby([
    'lat', 
    'lon', 
    'weather_bin_year',
    'weather_bin_month',
    'weather_bin_day'
]).mean().reset_index()

data_daily_mean.reset_index(drop=True, inplace=True)

# Reassign fractional ignition values a '1'
ignitions = data_daily_mean[data_daily_mean['ignition'] > 0].copy()
no_ignitions = data_daily_mean[data_daily_mean['ignition'] == 0].copy()
ignitions['ignition'] = 1

data_daily_mean = ignitions.append(no_ignitions)

In [ ]:
# Loop over dataset list, run k-fold cross validation on stratified training sample
# from each. Report raw scores.

dataset_descriptions = [
    "Raw data",
    "Rolling window",
    "Daily average"
]

datasets = (
    raw_data,
    data_rolling_window,
    data_daily_mean
)

classifier = XGBClassifier(
    n_jobs = max_jobs,
    scale_pos_weight = class_ratio,
    random_state = rand_seed
)

# Start empty dataframe to hold results
dataset_comparison_results = pd.DataFrame(columns=[
    'Data type',
    'Raw scores',
])

# Loop on datasets
for dataset, description in zip(datasets, dataset_descriptions):
    
    cross_val_scores = []
    
    for i in range(repetitions):
  
        # Draw train-test sample from dataset
        X_train, X_test, y_train, y_test = make_train_test_sample(
            dataset, 
            sample_size, 
            train_test_split_ratio, 
            rand_seed
        )

        # Scale weather variables
        X_train, X_test = scale_weather_variables(weather_variables, X_train, X_test)

        # Crossvalidate on training data
        scores = cross_validate_classifier(
            classifier, 
            X_train, 
            y_train, 
            cv_folds, 
            scoring_func
        )
        
        cross_val_scores.append(scores)

    # Store raw scores
    dataset_comparison_results = dataset_comparison_results.append(pd.Series([
        description,
        cross_val_scores,
    ], index=dataset_comparison_results.columns), ignore_index=True)

In [ ]:
# Plot comparision of datasets by score
fig = plt.figure(1, figsize=(5, 5))
ax = fig.add_subplot(111)

bp = ax.boxplot(dataset_comparison_results['Raw scores'])
ax.set_xticklabels(dataset_comparison_results['Data type'])
plt.xlabel('Dataset')
plt.ylabel('CV {} score'.format(scoring_func_name))
plt.title('Dataset comparision')
plt.xticks(rotation=45)
plt.show()

Ok, looks like the daily averages are the winner, though the rolling window is close. Before moving on, let's see what happens if we try to optimize the rolling window width.

## Metaparameter optimization: rolling window width

The use of a rolling window introduces a possibly extremely important metaparameter: the rolling window width. Let's play with that next.

In [ ]:
# Loop over rolling window sizes, run k-fold cross validation on stratified 
# training sample for each. Report raw scores, false positive and false negative rates.

window_widths = range(2,50,2)

classifier = XGBClassifier(
    n_jobs = max_jobs,
    scale_pos_weight = class_ratio,
    random_state = rand_seed
)

# Start empty dataframe to hold results
model_scores = pd.DataFrame(columns=[
    'Window width',
    'CV {} score'.format(scoring_func_name),
    'CV False positive rate',
    'CV False negative rate'
])

# Loop over window width
for window_width in window_widths:
    
    # Roll dataset with window 
    data_rolling_window = raw_data.iloc[::-1]
    data_rolling_window = data_rolling_window.groupby(['lat', 'lon']).rolling(window_width, on="weather_bin_time").mean()
    data_rolling_window.reset_index(drop=True, inplace=True)
    
    ignitions = data_rolling_window[data_rolling_window['ignition'] > 0].copy()
    no_ignitions = data_rolling_window[data_rolling_window['ignition'] == 0].copy()
    ignitions['ignition'] = 1

    dataset = ignitions.append(no_ignitions)
    
    # Draw train-test sample from dataset
    X_train, X_test, y_train, y_test = make_train_test_sample(
        dataset, 
        sample_size, 
        train_test_split_ratio, 
        rand_seed
    )
    
    # Scale weather varaibles
    X_train, X_test = scale_weather_variables(weather_variables, X_train, X_test)
    
    # Run cross validation on traing data
    cross_val_scores = cross_validate_classifier(
        classifier, 
        X_train, 
        y_train, 
        cv_folds, 
        scoring_func
    )

    score = mean(cross_val_scores)
    
    # Run cross validation with false positive rate
    cross_val_FP_rates = cross_validate_classifier(
        classifier, 
        X_train, 
        y_train, 
        cv_folds, 
        make_scorer(FP_rate_scorer)
    ) 

    cross_val_FP_rate = mean(cross_val_FP_rates)

    # Run cross validation with false negative rate
    cross_val_FN_rates = cross_validate_classifier(
        classifier, 
        X_train, 
        y_train, 
        cv_folds, 
        make_scorer(FN_rate_scorer)
    ) 

    cross_val_FN_rate = mean(cross_val_FN_rates)

    # Store results
    model_scores = model_scores.append(pd.Series([
        window_width,
        np.round(score,3), 
        np.round(cross_val_FP_rate,3), 
        np.round(cross_val_FN_rate,3)
    ], index=model_scores.columns), ignore_index=True)

In [ ]:
# Plot false positive and negative rates as a function of window size
plt.subplots(1,2,figsize=(12,12))

plt.subplot(1, 2, 1)
plt.scatter(
    model_scores['Window width'], 
    model_scores['CV False negative rate'],
    color = "darkred",
    label = 'False negatives'
)
plt.xlabel('Window width (hr.)')
plt.ylabel('Cross validation rate')
plt.title('Window width vs. false negatives')

plt.subplot(1, 2, 2)
plt.scatter(
    model_scores['Window width'], 
    model_scores['Average CV rate'],
    color = "darkblue",
    label = 'False positives'
)
plt.xlabel('Window width (hr.)')
plt.ylabel('Average CV rate')
plt.title('Window width vs. false positives')

plt.tight_layout()
plt.show()

In [ ]:
# Plot score as a function of window size
plt.scatter(
    model_scores['Window width'], 
    model_scores['CV {} score'.format(scoring_func_name)],
    color = "darkgreen",
    label ='CV {} score'.format(scoring_func_name),
)
plt.xlabel('Window width (hr.)')
plt.ylabel('CV {} score'.format(scoring_func_name))
plt.title('Window width vs. {} score'.format(scoring_func_name))
plt.show()

## Hyperparameter optimization: class weight

One of the challenges inherent in this dataset is imbalanced classes. An appripriate class weight can help aleviate this. Also, one of the strange consequences of increasing the window width is an increase in the number of observations labeled fire. Now that we have the window width metaparameter nailed down, let's optimize the class weight.

In [ ]:
# Define class weight search space
class_weights = np.logspace(-2, 3, num=25, base=10)

# Run cross validation on each class weight, report score
# false positive and false negative rates
model_scores = tune_class_weight(
    class_weights,
    max_jobs,
    rand_seed,
    data_daily_mean,
    repitions
)

In [ ]:
# Plot false positive rate, false negative rate and score
# as a function of class weight
plt.subplots(1,2,figsize=(12,5))

plt.subplot(1, 2, 1)
plt.scatter(
    np.log10(model_scores['Class weight']), 
    model_scores['CV False positive rate'],
    color = "darkblue",
    label = 'False positives'
)
plt.scatter(
    np.log10(model_scores['Class weight']), 
    model_scores['CV False negative rate'],
    color = "darkred",
    label = 'False negatives'
)
plt.xlabel('Log10 class weight ratio')
plt.ylabel('Cross validation rate')
plt.title('Class weight vs. false positives & negatives')
plt.legend()

plt.subplot(1, 2, 2)
plt.scatter(
    np.log10(model_scores['Class weight']), 
    model_scores['CV {} score'.format(scoring_func_name)],
    color = "darkgreen",
    label ='CV {} score'.format(scoring_func_name),
)
plt.xlabel('Log10 class weight ratio')
plt.ylabel('CV {} score'.format(scoring_func_name))
plt.title('Class weight vs. {} score'.format(scoring_func_name))

plt.tight_layout()
plt.show()


Interesting. We can use larger class weights to drive down the false negative rate at the expense of false positives. Looking at the clearly sigmoidal score curve, the optimum value seems to be between 1 and 10.

In [ ]:
# Pick and store the class wight which gives the highest score
model_scores = model_scores.sort_values(['CV {} score'.format(scoring_func_name)], ascending=[0])
model_scores.reset_index(drop=True, inplace=True)
class_weight = model_scores.loc[0,'Class weight']
print("{} optimized class weight: {}".format(scoring_func_name,np.round(class_weight,2)))

**Note:** Important to keep in mind here that we can tune our false positive/false negative rates easily with this hyperparameter.

## Hyperparameter tuning: learning rate and tree count

First, let's try taking one sample of the full dataset and then using RandomizedSearchCV to try and find the best values for learning rate and tree count.

In [ ]:
# Define classifier whith all the hyperparameters we have determined so far
classifier = XGBClassifier(
    n_jobs = classifier_jobs,
    scale_pos_weight = class_weight,
    random_state = rand_seed
)

# Set search space
param_dist = {
    'learning_rate': loguniform(0.0001, 1),
    'n_estimators': range(1,200),
    'max_depth': range(1, 21, 1),
    'gamma': np.linspace(0, 10, 101),
    'min_child_weight': loguniform(0.0001, 10),
    'reg_lambda': loguniform(0.0001, 10)
    
}

# Run randomsearchCV
best_model, random_search = tune_hyperparameters(
    classifier,
    param_dist, 
    X_train, 
    y_train, 
    optimization_jobs, 
    grid_search_iterations, 
    scoring_func
)

In [ ]:
# Extract results to dataframe
rand_search_results = pd.DataFrame(random_search.cv_results_).dropna()

# Make desnity plot showing the dependenc of score on
# learning rate and n_estimators
x = rand_search_results['param_n_estimators']
y = rand_search_results['param_learning_rate']
z = rand_search_results['mean_test_score']
xi, yi, zi = regularize_grid(x, y, z, plot_grid_resolution)

sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})

plt.contourf(xi, yi, zi, contourf_levels, cmap=plt.cm.Blues)
plt.xlabel("N estimators")
plt.ylabel("Learning rate")
plt.title("Effect of estimator count and \nlearning rate on score")
plt.colorbar()
plt.show()

Surprised by how 'rough' hyperparameter space is. Let's keep the winning numbers and save the scores to our log.

In [ ]:
best_learning_rate = rand_search_results.iloc[0]['param_learning_rate']
best_n_estimators = rand_search_results.iloc[0]['param_n_estimators']
print("{} optimized learning rate: {}".format(scoring_func_name,np.round(best_learning_rate,2)))
print("{} optimized n estimators: {}".format(scoring_func_name,np.round(best_n_estimators,2)))

best_depth = rand_search_results.iloc[0]['param_max_depth']
best_gamma = rand_search_results.iloc[0]['param_gamma']
print("{} optimized tree depth: {}".format(scoring_func_name,np.round(best_depth,2)))
print("{} optimized gamma: {}".format(scoring_func_name,np.round(best_gamma,2)))

best_min_child_weight = rand_search_results.iloc[0]['param_min_child_weight']
best_reg_lambda = rand_search_results.iloc[0]['param_reg_lambda']
print("{} optimized min. child weight: {}".format(scoring_func_name,np.round(best_min_child_weight,2)))
print("{} optimized L2 regularization: {}".format(scoring_func_name,np.round(best_reg_lambda,2)))

# Plot confusion matrix
class_names = np.array(['No ignition', 'Ignition'])
display_confusion_matrix(best_model, class_names, X_test, y_test)

## Hyperparameter tuning: tree depth and gamma

In [ ]:
# Define classifier whith all the hyperparameters we have determined so far
classifier = XGBClassifier(
    n_jobs = classifier_jobs,
    scale_pos_weight = class_weight,
    random_state = rand_seed,
    learning_rate = best_learning_rate,
    n_estimators = best_n_estimators
)

# Set search space
param_dist = {
    'max_depth': range(1, 21, 1),
    'gamma': np.linspace(0, 10, 101)
}

# Run randomsearchCV
best_model, random_search = tune_hyperparameters(
    classifier,
    param_dist, 
    X_train, 
    y_train, 
    optimization_jobs, 
    grid_search_iterations, 
    scoring_func
)

In [ ]:
# Extract results to dataframe
rand_search_results = pd.DataFrame(random_search.cv_results_).dropna()

# Make desnity plot showing the dependenc of score on
# learning rate and n_estimators
x = rand_search_results['param_max_depth']
y = rand_search_results['param_gamma']
z = rand_search_results['mean_test_score']
xi, yi, zi = regularize_grid(x, y, z, plot_grid_resolution)

sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})

plt.contourf(xi, yi, zi, contourf_levels, cmap=plt.cm.Blues)
plt.xlabel("Tree depth")
plt.ylabel("Gamma")
plt.title("Effect of tree depth and gamma on score")
plt.colorbar()
plt.show()

Again store winning parameter for later use.

In [ ]:
best_depth = rand_search_results.iloc[0]['param_max_depth']
best_gamma = rand_search_results.iloc[0]['param_gamma']
print("F1 optimized tree depth: {}".format(np.round(best_depth,2)))
print("F1 optimized gamma: {}".format(np.round(best_gamma,2)))

## Feature importance

Now that we have our hyperparameter nailed down, let't take a look at feature importance. First thing to do is train our optimized model on the whole dataset.

In [ ]:
# Train with tuned hyperparameters
classifier = XGBClassifier(
    n_jobs = max_jobs,
    scale_pos_weight = class_weight,
    learning_rate = best_learning_rate,
    n_estimators = best_n_estimators,
    random_state = rand_seed,
    max_depth = best_depth,
    gamma = best_gamma
)

# Stratified test train split the whole dataset
X_train, X_test, y_train, y_test = make_train_test_sample(
    data_daily_mean, 
    len(data_daily_mean), 
    train_test_split_ratio, 
    rand_seed
)

# Scale weather varibles
X_train, X_test = scale_weather_variables(weather_variables, X_train, X_test)

# Train model
classifier.fit(X_train, y_train)

# Plot confusion matrix
class_names = np.array(['No ignition', 'Ignition'])
display_confusion_matrix(classifier, class_names, X_test, y_test)

In [ ]:
false_neg_rate, false_pos_rate = calc_false_neg_pos_rate(classifier, X_test, y_test)
print("False negative rate: {}".format(np.round(false_neg_rate, 2)))
print("False positive rate: {}".format(np.round(false_pos_rate, 2)))

In [ ]:
plot_relative_feature_importance(classifier, data_daily_mean, X_test, 20)

Looks like only our top 5 or so features are actually contributing anything. Let's retrain using only those features.

In [ ]:
# grab top n feature names
feature_names = np.array(list(X_train))
importances = classifier.feature_importances_
indices = np.argsort(importances)[::-1]
top_n_features = feature_names[indices[0:5]]

# rebuild training and test sets with feature subset
X_train, X_test, y_train, y_test = make_train_test_sample(
    data_daily_mean, 
    len(data_daily_mean), 
    train_test_split_ratio, 
    rand_seed
)

X_train, X_test = scale_weather_variables(weather_variables, X_train, X_test)

X_train_subset = X_train[top_n_features]
X_test_subset = X_test[top_n_features]

# Train model
classifier.fit(X_train_subset, y_train)

# Plot confusion matrix
class_names = np.array(['No ignition', 'Ignition'])
display_confusion_matrix(classifier, class_names, X_test_subset, y_test)

In [ ]:
false_neg_rate, false_pos_rate = calc_false_neg_pos_rate(classifier, X_test_subset, y_test)
print("False negative rate: {}".format(np.round(false_neg_rate, 2)))
print("False positive rate: {}".format(np.round(false_pos_rate, 2)))

## Final model: robustness
Let's see how our final model holds up to repeated sampling - are the scores variable? or are the consistent across samples?

In [ ]:
train_scores = list()
test_scores = list()
false_neg_rates = list()
false_pos_rates = list()

sample_size = 5000

for i in range(0, 50):
    # rebuild training and test sets with feature subset
    X_train, X_test, y_train, y_test = make_train_test_sample(
        data_daily_mean, 
        sample_size, 
        train_test_split_ratio, 
        rand_seed
    )
    
    X_train, X_test = scale_weather_variables(weather_variables, X_train, X_test)
    
    X_train_subset = X_train[top_n_features]
    X_test_subset = X_test[top_n_features]

    classifier.fit(X_train_subset, y_train)

    y_pred = classifier.predict(X_test_subset)
    
    train_score = f1_score(y_train, classifier.predict(X_train_subset))
    test_score = f1_score(y_test, classifier.predict(X_test_subset))
        
    false_neg_rate, false_pos_rate = calc_false_neg_pos_rate(classifier, X_test_subset, y_test)
    
    train_scores.append(train_score)
    test_scores.append(test_score)
    false_neg_rates.append(false_neg_rate)
    false_pos_rates.append(false_pos_rate)

In [ ]:
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})

ax = sns.kdeplot(train_scores, label="Training data", shade=True, color="darkblue")
ax = sns.kdeplot(test_scores, label="Test data", shade=True, color="darkred")
ax.set_title("{} score distributions, repeated samples".format(scoring_func_name))
ax.set(xlabel='{} score'.format(scoring_func_name), ylabel='Density')
plt.legend()
plt.legend(loc='upper left')

plt.show()

In [ ]:
ax = sns.kdeplot(false_neg_rates, label="False negative", shade=True, color="darkblue")
ax = sns.kdeplot(false_pos_rates, label="False positive", shade=True, color="darkred")
ax.set_title("False positive and negative rates\nrepeated samples")
ax.set(xlabel='Rate', ylabel='Density')
plt.legend()

plt.show()

In [ ]:
plt.scatter(false_neg_rates, false_pos_rates, s=5)
plt.xlabel("False negative rate")
plt.ylabel("False positive rate")
plt.title("False positive vs false negative rate\nrepeated samples")

plt.show()

## Conclusion

Overall, XGBoost was selected as the best classifier for this problem. The initial model performance was improved by optimizing dataset smoothing and hyperparameters. Then feature importance was investigated and lowly important features pruned.

## Future work

The current model can likely be further improved in four ways:
1. Apply neural net based machine learning.
2. Add more features to the data such as: elevation, total fires at a location etc.
3. Reframe the problem as regression by calculating fire probability, rather than using a binary 'yes'-'no' approach.
4. Implement 'fully stratified sampling' i.e. choose samples for training, validation and testing such that the distribution of all factors in the sample match the parent dataset as closely as possible.